## Loading loan data

In [19]:
from utils.dataload import load_loan_data

In [20]:
loan_df = load_loan_data("./data/1.csv")

## IR

In [24]:
import modules.IR as IR

## L

In [16]:
import modules.L as L

In [18]:
a = L.generate_CPR_HullWhite()

b = 'b'
h = 'h'

FileNotFoundError: [Errno 2] No such file or directory: 'Dates.csv'

## CF

In [14]:
import modules.CF as CF

In [23]:
x = CF.Loan_Portfolio(0.10, loan_df)
y = x.cash_flows_aggregate_for_portfolio()
z = x.cash_flow_tranchwise(loan_df)
print(y["total_payments"])

NameError: name 'h' is not defined

## A

In [21]:
import modules.A as A

In [22]:
price = A.MBS_pricing(x, y, z)
price.import_MBS_Cashflows()
price.caculate_tranch_value()
price.plot_mbs_value()

NameError: name 'x' is not defined